In [65]:
import pandas as pd
# read the dataset in from the excel file
df = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
orders = pd.read_csv('https://raw.githubusercontent.com/raclan/CIS_4496_Project/main/code/notebooks/df_ct.csv')
# show the first few rows
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [41]:
orders.head()

,customer_id,articles
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[663713001, 541518023, 663713001, 578020002, 7..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[505221004, 685687003, 685687004, 685687001, 5..."
2,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,"[688873012, 501323011, 598859003, 688873020, 6..."
3,0008968c0d451dbc5a9968da03196fe20051965edde741...,"[531310002, 529841001, 662980002, 496111003, 5..."
4,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,"[501820043, 501820043, 674681001, 671505001, 6..."


In [42]:
type(orders['articles'][0])

str

In [43]:
from ast import literal_eval
orders['articles'] = orders['articles'].apply(literal_eval)

In [44]:
type(orders['articles'][0])

list

In [45]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

orders_1hot = te.fit_transform(orders['articles'])

In [46]:
# convert orders_1hot to a DataFrame
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
# Inspect the first 5 rows of the DataFrame
orders_1hot.head()

,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,...,947060001,947168001,947509001,947934001,949198001,949551001,949551002,952267001,952938001,956217002
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [55]:
from mlxtend.frequent_patterns import fpgrowth
# the moment we have all been waiting for (again)
orders_fp = fpgrowth(orders_1hot, min_support=0.0001, max_len=2,
                use_colnames=True)


In [56]:
from mlxtend.frequent_patterns import association_rules
ar = association_rules(orders_fp, 
                       metric="lift", 
                       min_threshold=0.01)

In [57]:
ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(351484002),(372860002),0.021793,0.031018,0.001211,0.055556,1.791099,0.000535,1.025981
1,(372860002),(351484002),0.031018,0.021793,0.001211,0.039033,1.791099,0.000535,1.017941
2,(751471001),(351484002),0.023984,0.021793,0.001441,0.060096,2.757587,0.000919,1.040752
3,(351484002),(751471001),0.021793,0.023984,0.001441,0.066138,2.757587,0.000919,1.045139
4,(351484002),(685687004),0.021793,0.028711,0.001153,0.052910,1.842821,0.000527,1.025550
...,...,...,...,...,...,...,...,...,...
12529525,(593758002),(835008009),0.000115,0.004728,0.000115,1.000000,211.524390,0.000115,inf
12529526,(593758002),(687524012),0.000115,0.005650,0.000115,1.000000,176.989796,0.000115,inf
12529527,(687524012),(593758002),0.005650,0.000115,0.000115,0.020408,176.989796,0.000115,1.020716
12529528,(851530002),(739092020),0.002364,0.000115,0.000115,0.048780,423.048780,0.000115,1.051161


In [59]:
def predict(antecedent, rules, max_results= 12):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]

In [69]:
preds = predict({df['article_id'][1]}, ar)
preds

2329    351484002
2331    399223001
2333    706016002
2335    770346001
2337    640156004
2339    253448003
2341    572797002
2343    372860001
2345    634775005
2347    678861003
2349    663519001
2351    775996012
Name: consequents, dtype: int64